In [1]:
import pgserver
db = pgserver.get_server(r'C:\Users\paul_ip\Documents\GitHub\hk-address-translate\pgdata2')

In [2]:
print(db.psql('create extension vector'))
db_uri = db.get_uri()
print(db_uri)


postgresql://postgres:@127.0.0.1:62589/postgres


In [3]:
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import create_engine
import sqlalchemy as sql
dburi = db.get_uri(database='mydb')
display(dburi)
if not database_exists(dburi):
    create_database(dburi)
engine = create_engine(dburi)

'postgresql://postgres:@127.0.0.1:62589/mydb'

In [14]:
import pandas as pd
df = pd.read_csv(r'C:\Users\paul_ip\Documents\GitHub\hk-address-translate\buildings_table.csv')
df.to_sql('buildings', engine, if_exists='replace', index=False)

C:\Users\paul_ip\AppData\Local\Temp\ipykernel_15240\3158244175.py:2: DtypeWarning: Columns (15,16,30,31,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\paul_ip\Documents\GitHub\hk-address-translate\buildings_table.csv')


296

In [6]:
import pandas as pd
#select the first 5 rows
sql_string = "SELECT * FROM buildings LIMIT 5;"
result = pd.read_sql_query(sql_string, engine)
print(result)

   northing   easting  latitude  longitude chi_Region chi_ChiDistrict  \
0  816062.0  833501.0  22.28344  114.15002         香港             中西區   
1  815940.0  833802.0  22.28234  114.15294         香港             中西區   
2  816031.0  833589.0  22.28316  114.15088         香港             中西區   
3  815791.0  833654.0  22.28099  114.15151         香港             中西區   
4  816328.0  832527.0  22.28584  114.14058         香港             中西區   

  chi_ChiStreet_StreetName chi_ChiStreet_BuildingNoFrom  \
0                    必列者士街                           58   
1                      嘉咸街                           46   
2                      城皇街                           17   
3                      太子台                           12   
4                      西邊街                           15   

  chi_ChiEstate_EstateName                           eng_EngEstate_EstateName  \
0                英皇書院同學會小學  KING'S COLLEGE OLD BOYS' ASSOCIATION PRIMARY S...   
1                     None                  

In [ ]:
# ...existing code...

# Add a new cell after the existing index_str cell to create the index
# First, add a generated column for the concatenated English address, grouping number components
alter_table_sql = """
ALTER TABLE buildings 
ADD COLUMN eng_full_address TEXT GENERATED ALWAYS AS (
    COALESCE(buildings.eng_Region, '') || ' ' ||
    COALESCE(buildings.eng_EngDistrict, '') || ' ' ||
    COALESCE(buildings.eng_EngEstate_EstateName, '') || ' ' ||
    COALESCE(buildings.eng_EngEstate_EngPhase_PhaseName, '') || ' ' ||
    COALESCE(buildings.eng_EngEstate_EngPhase_PhaseNo, '') || ' ' ||
    COALESCE(buildings.eng_EngBlock_BlockDescriptor, '') || ' ' || COALESCE(buildings.eng_EngBlock_BlockNo, '') || ' ' ||  -- Group block descriptor and block no
    COALESCE(buildings.eng_BuildingName, '') || ' ' ||
    COALESCE(buildings.eng_EngStreet_BuildingNoFrom, '') || ' ' || COALESCE(buildings.eng_EngStreet_BuildingNoTo, '') || ' ' ||  -- Group building numbers
    COALESCE(buildings.eng_EngStreet_StreetName, '') || ' ' ||
    COALESCE(buildings.eng_EngVillage_VillageName, '') || ' ' ||
    COALESCE(buildings.eng_EngVillage_BuildingNoFrom, '') || ' ' || COALESCE(buildings.eng_EngVillage_BuildingNoTo, '') || ' ' ||  -- Group village building numbers
    COALESCE(buildings.eng_EngVillage_LocationName, '') || ' ' ||
    COALESCE(buildings.eng_EngStreet_LocationName, '')
) STORED;
"""

# Execute the ALTER TABLE
with engine.connect() as conn:
    conn.execute(sql.text(alter_table_sql))
    conn.commit()

# Then, create a GIN index on the tsvector of the full address
create_index_sql = """
CREATE INDEX idx_eng_address_fts ON buildings USING GIN (to_tsvector('english', eng_full_address));
"""

# Execute the CREATE INDEX
with engine.connect() as conn:
    conn.execute(sql.text(create_index_sql))
    conn.commit()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "eng_region" does not exist
HINT:  Perhaps you meant to reference the column "buildings.eng_Region".

[SQL: 
ALTER TABLE buildings 
ADD COLUMN eng_full_address TEXT GENERATED ALWAYS AS (
    COALESCE(eng_Region, '') || ' ' ||
    COALESCE(eng_EngDistrict, '') || ' ' ||
    COALESCE(eng_EngEstate_EstateName, '') || ' ' ||
    COALESCE(eng_EngEstate_EngPhase_PhaseName, '') || ' ' ||
    COALESCE(eng_EngEstate_EngPhase_PhaseNo, '') || ' ' ||
    COALESCE(eng_EngBlock_BlockDescriptor, '') || ' ' || COALESCE(eng_EngBlock_BlockNo, '') || ' ' ||  -- Group block descriptor and block no
    COALESCE(eng_BuildingName, '') || ' ' ||
    COALESCE(eng_EngStreet_BuildingNoFrom, '') || ' ' || COALESCE(eng_EngStreet_BuildingNoTo, '') || ' ' ||  -- Group building numbers
    COALESCE(eng_EngStreet_StreetName, '') || ' ' ||
    COALESCE(eng_EngVillage_VillageName, '') || ' ' ||
    COALESCE(eng_EngVillage_BuildingNoFrom, '') || ' ' || COALESCE(e

In [11]:
query = """
select to_tsvector('BLK T6 YAT HONG MANSN 47 TAI HONG ST SAI WAN HO, HK ')"""
result = pd.read_sql_query(query, engine)
print(result)

                                         to_tsvector
0  '47':6 'blk':1 'hk':13 'ho':12 'hong':4,8 'man...


In [16]:
#terminate the server connection
db.cleanup()